<a href="https://colab.research.google.com/github/RajArPatra/Dimentionality-Reduction/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the dataset

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1Nl-RU5HggCSWWqINN4gbxHAVzER6T2Po&export=download
!gzip -d /content/HIGGS_6M.csv.gz

Downloading...
From: https://drive.google.com/u/0/uc?id=1Nl-RU5HggCSWWqINN4gbxHAVzER6T2Po
To: /content/HIGGS_6M.csv.gz
1.56GB [00:08, 180MB/s]


Import Libraries

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import pandas as pd
import torchvision
import os
import numpy as np
from skimage import io,transform as ts
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import imgaug as ia
import imgaug.augmenters as iaa
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#from bhtsne import tsne
from torch.autograd import Variable

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

Train and Test Split Preparation

In [ ]:
import pandas as pd 
import numpy as np
df= pd.read_csv('/content/HIGGS_6M.csv')

df=df.replace([np.inf, -np.inf],np.nan)
#print(df.isnull().sum())
df1=df.iloc[:-500000]
df2=df.iloc[-500000:]

np.save('train.npy', df1.to_numpy(),True)
np.save('test.npy', df2.to_numpy(),True)
del df,df1,df2

Dataset and Dataloader

In [ ]:
class data1(Dataset):
    def __init__(self, train=True,device='cpu'):
      self.train=train
      self.scaler=MinMaxScaler()
      
      #self.scaler.fit(np.load('train_num.npy',allow_pickle=True)[:,-1].reshape(-1, 1))
      self.device=device
      #x_test = np.load(f,allow_pickle=True)
      #y_test= np.load('/content/y_test_pred.npy',allow_pickle=True) 
      #z=int(0.9*X_train.shape[0])
      
     
      if train:
        self.data= np.load('train_auto.npy',allow_pickle=True)
        self.data_x=self.data[:,1:]
        
        self.data_y=self.data[:,0]
      
      else:
        self.data= np.load('test_auto.npy',allow_pickle=True)
        self.data_x=self.data[:,1:]
        
        self.data_y=self.data[:,0]
      
     
      
    def __getitem__(self, idx):
     
      #X=scaler.fit_transform(X.reshape(1, -1))
      X=torch.from_numpy(self.data_x[idx]).float()
      
      
      
      '''if self.train:
        y=torch.tensor(self.map[int(self.data_y[idx])])
      else:'''
      
      y=torch.tensor(self.data_y[idx]).long()
      
     
      return X.to(self.device), y.to(self.device)
      
    def __len__(self):
      
      return self.data.shape[0]

In [ ]:
if torch.cuda.is_available():
  device='cuda'
else :
   device='cpu'
device

'cuda'

In [ ]:
train_dataset=data1(train=True,device=device)
test_dataset=data1(train=False,device=device)


In [ ]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=256,
                                           shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=256, 
                                           shuffle=False)

Model Architecture

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self,D_in,H=50,H2=12,latent_dim=3):
        
        #Encoder
        super(Autoencoder,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
#         # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))


        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        # self.decode(z) ist später recon_batch, mu ist mu und logvar ist logvar
        return self.decode(z), mu, logvar

In [ ]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    # x_recon ist der im forward im Model erstellte recon_batch, x ist der originale x Batch, mu ist mu und logvar ist logvar 
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

Helper function to Initialise weight

In [ ]:
from torch.nn import init
def init_weights(net, init_type='normal', gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                init.normal_(m.weight.data, 0.0, gain)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=gain)
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:
            init.normal_(m.weight.data, 1.0, gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)

Save CheckPoint

In [ ]:
def save_checkpoint(model, save_folder, name, epoch, opt, best_model=False):

    if best_model:
        save_path = "{}/{}/best_model.th".format(
            save_folder, name)
    else:
        save_path = "{}/{}/epoch_{}.th".format(save_folder,
                                               name, epoch)

    if not os.path.exists(save_folder + "/" + name):
        os.makedirs(save_folder + "/" + name)

    print("SAVE MODEL to", save_path)


    param_dict = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict()}
        

    torch.save(param_dict, save_path)

Load CheckPoint

In [ ]:
def load_checkpoint(model, save_folder, model_name, opt,epoch,device ,best_model=False):
      if best_model:
        load_path = "{}/model_{}/best_model.th".format(save_folder, model_name)
      else:
        load_path = "{}/{}/epoch_{}.th".format(save_folder,
                                                model_name, epoch)

      checkpoint = torch.load(load_path)

      epoch = checkpoint['epoch']
      

      model =model # code for model init
      model.load_state_dict(checkpoint['model_state_dict'])
      model=model.to(device)

      opt =opt #code for opt init
      opt.load_state_dict(checkpoint['optimizer_state_dict'])
      #opt.param_groups[0]['lr'] = checkpoint['optimizer_params']['lr']
      return model, opt, epoch   


In [ ]:
D_in = 28
H = 50
H2 = 12
model = Autoencoder(D_in, H, H2).to(device)
init_weights(model,'kaiming')
#model.to(device)
#model.apply(weights_init_uniform_rule)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

criterion=customLoss()

initialize network with kaiming


Train

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data,_) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = criterion(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
#        if batch_idx % log_interval == 0:
#            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                epoch, batch_idx * len(data), len(trainloader.dataset),
#                       100. * batch_idx / len(trainloader),
#                       loss.item() / len(data)))
    if epoch % 1 == 0:        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_loader.dataset)))
        train_losses.append(train_loss / len(train_loader.dataset))

train_losses=[]
for i in range(20):
  
  train(i+lst_epoch+1)
  #scheduler.step()
  if ((i+lst_epoch+1)%5) ==0:
    save_checkpoint(model,'/content/drive/MyDrive/Model_files','dim_reduction',(i+lst_epoch+1),optimizer)
  elif i==19:
    save_checkpoint(model,'/content/drive/MyDrive/Model_files','dim_reduction',(i+lst_epoch+1),optimizer)

Save Dimention reduced Features

In [ ]:
mu_output = []
logvar_output = []

with torch.no_grad():
    for i, (data,_) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)

            
            mu_tensor = mu   
            mu_output.append(mu_tensor)
            mu_result = torch.cat(mu_output, dim=0)
            
            logvar_tensor = logvar   
            logvar_output.append(logvar_tensor)
            logvar_result = torch.cat(logvar_output, dim=0)

            